# Dataset Cleaning & Feature Engineering using Titanic dataset
Pclass - Passenger Class  (1 = 1st; 2 = 2nd; 3 = 3rd)<br>
sex - Gender<br>
sibsp - Number of siblings/Spouses<br>
parch - Number of parents/children<br>
fare - travel fare<br>
Embarked - (C = Cherbourg; Q = Queenstown; S = Southampton)<br>
boat - Life boat<br>
body - Body identification number<br>
home.dest - Destination<br>
ticket - Ticket number<br>
cabin - Cabin number<br>
name - Passenger name<br>
survived - (0: No, 1: Yes)<br>

In [ ]:
# Set up the environment for using pyspark
import findspark

findspark.init()

In [ ]:
import pyspark

In [ ]:
# Create Spark Session
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors

In [ ]:
# Create Application Context
spark = SparkSession.builder.appName("Titanic Dataset").getOrCreate()
sc = spark.sparkContext

### Data ingestion

In [ ]:
# Read the dataset file which is in csv - comma separated values format
sdf = spark.read.format('csv').options(header='true', inferSchema='true').load('datasets/titanic3.csv')

In [ ]:
import pandas as pd

In [ ]:
df_pd = sdf.toPandas()

### Data Exploration

In [ ]:
df_pd.head()

In [ ]:
sdf.printSchema()

In [ ]:
to_drop = ['boat', 'body', 'home.dest', 'ticket', 'cabin', 'name']

In [ ]:
# drop interface on dataframe requires individual string
# to pass a list convert the argument as *args
sdf = sdf.drop(*to_drop)

In [ ]:
sdf.toPandas().head()

In [ ]:
print('Shape: ({},{})'. format(sdf.count(), len(sdf.columns)))

In [ ]:
# if the survived has null value in it then drop the sample
from pyspark.sql.functions import col
sdf = sdf.where(col("survived").isNotNull())

In [ ]:
print('Shape: ({},{})'. format(sdf.count(), len(sdf.columns)))

### Show columns with null values in the dataframe

In [ ]:
from pyspark.sql.functions import col
def show_null_value_count(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        if(nullRows > 0):
            temp = k,nullRows
            null_columns_counts.append(temp)
    spark.createDataFrame(null_columns_counts, ['Column_With_Null_Value', 'Null_Values_Count']).show()

In [ ]:
show_null_value_count(sdf)

In [ ]:
# Check the count - indicator of columns with missing values
# Check for non numeric values in columns e.g. sex, emabarked, name
sdf.describe().toPandas().head()

In [ ]:
# Find distinct values in embarked
sdf.select('embarked').distinct().count()

In [ ]:
sdf.crosstab('age', 'sex').show()

In [ ]:
sdf.groupby('age').count().show()

### Change gender to numeric values

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='sex', outputCol="sex_numeric").fit(sdf)
sdf = indexer.transform(sdf)
sdf.select('sex', 'sex_numeric').show()

In [ ]:
sdf.columns

In [ ]:
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCols=['age'], outputCols=['age_filled'])
model = imputer.fit(sdf)
sdf = model.transform(sdf)
sdf.show()

In [ ]:
imputer = Imputer(inputCols=['fare'], outputCols=['fare_filled'])
model = imputer.fit(sdf)
sdf = model.transform(sdf)

In [ ]:
sdf.columns

### Embarked Feature processing
1. replace null values with most occuring
2. create indexer to replace the string values
3. apply onehotencoderestimator on the indexed values

In [ ]:
sdf.groupBy('embarked').count().orderBy('count').show()

In [ ]:
sdf = sdf.na.fill({"embarked" :'S'})

In [ ]:
sdf.groupBy('embarked').count().orderBy('count').show()

### Now apply indexer and OneHotEncoding to embarked

In [ ]:
indexer = StringIndexer(inputCol='embarked', outputCol="embarked_numeric").fit(sdf)
sdf = indexer.transform(sdf)
sdf.select('embarked', 'embarked_numeric').show()

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["embarked_numeric"],
                                 outputCols=["embarked_Vec"])
model = encoder.fit(sdf)
sdf = model.transform(sdf)
sdf.show()

In [ ]:
print('Shape: ({},{})'. format(sdf.count(), len(sdf.columns)))

In [ ]:
sdf.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler
fCols = ['pclass', 'sex_numeric', 'age_filled', 'sibsp', 'parch', 'fare_filled', 'embarked_Vec']
feature = VectorAssembler(inputCols = fCols, outputCol = 'features')
data = feature.transform(sdf)

In [ ]:
data.printSchema()

# Now Data is clean